# Parcial Practico 1

## Ernesto Camacho Arroyave - MIND 2022-1

In [1]:
import pandas as pd

**1) Describa de forma clara y precisa el dataset que le fue asignado. Complementando con la dimensionalidad, la cantidad de atributos, la descripción de clase, cómo utilizarlo
en un problema de clasificación, etc.**

- El dataset que me correspondio es llamado **Car Evaluation**, se encuentra en: https://archive.ics.uci.edu/ml/datasets/car+evaluation .
- Segun la informacion que provee la misma pagina podemos ver: 
    - Que es un dataset multivariado, es decir que tiene distintos tipos de datos 
    - Que tiene un numero de 1728 entradas 
    - Que tiene un total de 6 atributos y una clase:
        - buying el precio de compra
        - maint el costo del mantenimiento
        - doors numero de puertas que tiene
        - persons capacidad de personas que tiene el vehiculo 
        - lug_boot la capacidad que tiene el bául 
        - safety que tan seguro es el vehiculo
        - class que tan aceptable es el vehiculo teniendo en cuenta las caracteristicas
    - Que segun la pagina no tiene valores que faltan, es decir es una dataset completo
- Dado que tiene una clase la idea seria crear un modelo que teniendo en cuenta las caracteristicas para determinar si el vehiculo vale la pena de comprar, rentar, o usar.

In [9]:
carDt= pd.read_csv('car.csv')
carDt.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [10]:
print(carDt.size)
print(carDt.shape)
print(carDt.ndim)

12096
(1728, 7)
2


**2) Comenzando con el preprocesamiento, usted debe decidir si necesario hacer normalización, discretización, selección de atributos, etc. La decisión de hacerlo o no, debe ser justificada.**

- Teniendo en cuenta los atributos, y que los valores que pueden tomar son relativamente pequeños, creo que discretizar los valores de los atributos cualitativos y volverlos cuantitavivos simplificara el analisis y perimitira crear un modelo que sea mas facil de adaptar.

- La discretización de los artributos dependera de los valores de los atributos, pero para cada uno se va a hacer una explicación:
    - buying: Este atributo puede tener valores **{vhigh, high, med, low}**, estos valores van a ser mapeados de la siguiente manera **{vhigh = 1, high = 2, med = 3, low =4}**.
    - maint: Este atributo puede tener valores **{vhigh, high, med, low}**, estos valores van a ser mapeados de la siguiente manera **{vhigh = 1, high = 2, med = 3, low =4}**.
    - doors: Es un atributo que por lo general es quantitativo y discreto, pero puede tomar un valor ***5-more***, para este caso lo vamos a pasar a solo numerico y que tome un valor de **5**
    - persons: Similar a doors, pero para este caso el valor es ***more***, en este caso lo pasaremos a 5 
    - lug_boot: Este atributo puede tener valores **{small, med, big}**, estos valores van a ser mapeados de la siguiente manera **{small = 1, med = 2, big = 3}**
    - safety: Este atributo puede tener valores **{low, med, high}**, estos valores van a ser mapeados de la siguiente manera **{low = 1, med = 2, high = 3}**
    - class: La clase puede tomar valores **{unacc, acc, good, vgood}**, estos valores van a ser mapeados de la siguiente manera **{unacc = 1, acc = 2, good = 3, vgood =4}**.